# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv
import joblib, pickle

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.data.datapath import DataPath
from azureml.data.dataset_factory import TabularDatasetFactory

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.24.0


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'mobile_price_prediction'

experiment=Experiment(ws, experiment_name)

In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

compute_cluster = 'automl-cluster'

try:
    comp_trget = ComputeTarget(workspace=ws, name=compute_cluster)
    print('Cluster already exists')
except ComputeTargetException:
    compute_confg = AmlCompute.provisioning_configuration(vm_size="STANDARD_D3_V2",max_nodes=4)
    comp_trget = ComputeTarget.create(ws,compute_cluster,compute_confg)

Cluster already exists


In [4]:
input_data = pd.read_csv("https://raw.githubusercontent.com/valeriapolozun/machine_learning_azure_capstone/803e04124532b8921a0b19d70aa71abaccf74a1f/Mobile_Price_Classification.csv")

In [ ]:
#input_data=pd.read_csv('Mobile_Price_Classification.csv')

In [5]:
input_data

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0.0,1,0,7,0.6,188.0,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1.0,0,1,53,0.7,136.0,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1.0,2,1,41,0.9,145.0,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0.0,0,0,10,0.8,131.0,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0.0,13,1,44,0.6,141.0,2,...,1208,1212,1411,8,2,15,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,996,1700,1.0,1.9,0,0,1,54.0,0.5,170,...,17,644,913,2121,14,8,15,1,1,0
2996,997,609,0.0,1.8,1,0,0,13.0,0.9,186,...,2,1152,1632,1933,8,1,19,0,1,1
2997,998,1185,0.0,1.4,0,1,1,8.0,0.5,80,...,12,477,825,1223,5,0,14,1,0,0
2998,999,1533,1.0,0.5,1,0,0,50.0,0.4,171,...,12,38,832,2509,15,11,6,0,1,0


In [6]:
# split data into training and test data
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(input_data,test_size=0.2)
train_data.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
2867,868,1670,0.0,0.5,0,1,1,33.0,0.9,127,...,3,185,807,1830,5,0,6,1,1,1
2388,389,1197,0.0,2.9,0,6,1,38.0,0.1,83,...,17,1516,1910,3525,13,9,16,1,0,0
1427,1489,0,2.1,1.0,0,0,9,0.4,169.0,3,...,923,1759,1663,9,3,14,1,0,1,1
2000,1,1043,1.0,1.8,1,14,0,5.0,0.1,193,...,16,226,1412,3476,12,7,2,0,1,0
38,790,0,2.0,1.0,16,1,11,0.3,87.0,6,...,347,730,3169,6,1,2,1,0,1,2


In [8]:
# Save training data to a csv file 
train_data.to_csv("data/mobile_price_traindata.csv", index=False)

In [10]:
data_store = ws.get_default_datastore()
data_store

{
  "name": "workspaceblobstore",
  "container_name": "azureml-blobstore-1896c5be-cb73-4775-bea9-388c806d3d0a",
  "account_name": "mlstrg141462",
  "protocol": "https",
  "endpoint": "core.windows.net"
}

In [11]:
data_store.upload(src_dir='data/',target_path='pricing', overwrite=True, show_progress=True)


Uploading an estimated of 1 files
Uploading data/mobile_price_traindata.csv
Uploaded data/mobile_price_traindata.csv, 1 files out of an estimated total of 1
Uploaded 1 files


$AZUREML_DATAREFERENCE_869eab05776a40eb9049b83fce08d37c

In [12]:
train_data = Dataset.Tabular.from_delimited_files(path = [(data_store, 'pricing/mobile_price_traindata.csv')])

In [13]:
train_data

{
  "source": [
    "('workspaceblobstore', 'pricing/mobile_price_traindata.csv')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ]
}

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [14]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 4,
    "primary_metric" : 'accuracy'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
                             compute_target=comp_trget,
                             task = "classification",
                             training_data=train_data,
                             label_column_name="price_range",   
                             enable_early_stopping= True,
                             featurization= 'auto',
                             n_cross_validations=5,
                             **automl_settings
)

In [15]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output = True)

Running on remote.
No run_configuration provided, running on automl-cluster with default configuration
Running on remote compute: automl-cluster
Parent Run ID: AutoML_88212ace-464b-4cbd-8e59-04f3e08e6a74

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing val

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [16]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more abo

{'runId': 'AutoML_88212ace-464b-4cbd-8e59-04f3e08e6a74',
 'target': 'automl-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-03-27T20:04:12.867695Z',
 'endTimeUtc': '2021-03-27T20:22:03.879409Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'automl-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"mobile_price_prediction","subscription_id":"3d1a56d2-7c81-4118-9790-f85d1acf0c77","resource_group":"aml-quickstarts-141462","workspace_name":"quick-starts-ws-141462","region":"southcentralus","compute_target":"automl-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":5,"y_

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [18]:
pip install xgboost==0.90

     |████████████████████████████████| 142.8 MB 25 kB/s /s eta 0:00:01
  Attempting uninstall: xgboost
    Found existing installation: xgboost 1.3.3
    Uninstalling xgboost-1.3.3:
      Successfully uninstalled xgboost-1.3.3
Note: you may need to restart the kernel to use updated packages.


In [20]:
best_automl, best_fit_model = remote_run.get_output()
print(best_fit_model)

ConfigException: ConfigException:
	Message: The model you attempted to retrieve requires 'xgboost' to be installed at '==0.90'. You have 'xgboost==1.3.3', please reinstall 'xgboost==0.90' (e.g. `pip install xgboost==0.90`) and rerun the previous command.
	InnerException: XGBoostError: [21:01:29] /mnt/xgboost/src/learner.cc:922: Check failed: header == serialisation_header_: 

  If you are loading a serialized model (like pickle in Python) generated by older
  XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version.  There's a simple script for helping
  the process. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for reference to the script, and more details about differences between saving model and
  serializing.


Stack trace:
  [bt] (0) /anaconda/envs/azureml_py36/lib/libxgboost.so(dmlc::LogMessageFatal::~LogMessageFatal()+0x7c) [0x7f016187b64c]
  [bt] (1) /anaconda/envs/azure

In [21]:
#TODO: Save the best model
joblib.dump(best_fit_model, 'best_fit_automl_model.pkl')

NameError: name 'best_fit_model' is not defined

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service